In [1]:
%cd /kaggle/input/

/kaggle/input


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostClassifier
import random
random.seed(42)
pd.set_option('display.max_columns', 120)

In [3]:
data_train = pd.read_parquet('/kaggle/input/hack228/data_train_fin.parquet')

In [4]:
data_train.region.fillna('None', inplace=True)
num_cols = ['key_rate']
for col in num_cols:
    data_train[col] = data_train[col].astype(str)
    data_train[col] = data_train[col].str.replace(',', '.')
    data_train[col] = data_train[col].astype(float)

/tmp/ipykernel_34/1270795207.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_train.region.fillna('None', inplace=True)


In [5]:
from sklearn.model_selection import train_test_split
X, y = data_train.drop(columns=['churn', 'npo_sum_min', 'npo_sum_mean', 'npo_sum_count', 'client_id', 'year', 'npo_account_id']), data_train['churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

In [9]:
X_train

,slctn_nmbr_x,npo_accnts_nmbr,pmnts_type,gender,age,clnt_cprtn_time_d,actv_prd_d,lst_pmnt_rcnc_d,oprtn_sum_per_qrtr,oprtn_sum_per_year,frst_pmnt,lst_pmnt,pmnts_sum,pmnts_nmbr,incm_sum,incm_per_qrtr,incm_per_year,mgd_accum_period,mgd_payment_period,phone_number,email,lk,assignee_npo,assignee_ops,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt,npo_oprtn_grp_min,npo_oprtn_grp_max,npo_oprtn_grp_count,quarter_sin,quarter_cos,latitude,longitude,key_rate,live,min_npo_sum_1_quarters_back,min_npo_sum_2_quarters_back,min_npo_sum_4_quarters_back,mean_npo_sum_3_quarters_back,sum_npo_sum_1_quarters_back,min_npo_oprtn_grp_1_quarters_back,slctn_nmbr_y,accnt_pnsn_schm,npo_account_id_count_open
790912,0,1,2,-1,49,323,302,35,1108.06,4588.58,366.13,420.77,3835.55,11,65.87,-0.00,28.77,0.0,0.0,-1,0,-1,-1,-1,БЕЛГОРОДСКАЯ ОБЛ,1,1,0,1,1.0,1.0,3.0,1.000000e+00,6.123234e-17,51.2222,38.5488,8.25,74.88,65.87,198.305,178.2050,318.355833,1077.27,0.0,0,354.0,1
282505,0,1,2,-1,46,2648,618,29,2212.66,10634.59,14431.65,968.33,23634.12,22,8002.53,-0.00,3046.93,0.0,0.0,-1,0,-1,-1,-1,ЛИПЕЦКАЯ ОБЛ,1,1,0,1,1.0,1.0,3.0,-1.000000e+00,-1.836970e-16,52.6186,39.5689,11.50,73.34,340.01,371.220,331.4775,639.249167,1721.36,1.0,0,354.0,1
2795250,0,1,1,-1,45,3676,3653,30,21109.61,46230.58,1028.15,5227.54,269060.31,130,99963.79,13596.82,13596.82,0.0,0.0,-1,0,-1,-1,-1,ХАБАРОВСКИЙ КРАЙ,1,1,0,1,0.0,1.0,4.0,-2.449294e-16,1.000000e+00,50.5271,137.5916,4.25,76.43,1242.18,962.350,1165.0325,3799.588056,6680.00,1.0,0,351.0,1
2526583,0,1,3,1,47,1973,1953,30,4992.45,29452.66,1932.36,1096.43,95781.94,67,14857.62,-0.00,6587.37,0.0,0.0,-1,0,-1,-1,-1,САРАТОВСКАЯ ОБЛ,1,1,0,1,1.0,1.0,3.0,-1.000000e+00,-1.836970e-16,51.9843,48.6775,7.00,68.24,812.11,1094.035,1056.0375,1898.275000,6329.07,1.0,0,351.0,1
3058438,0,2,1,-1,53,1696,1615,30,6015.72,14561.52,219.41,1555.30,50484.52,60,6480.85,2652.50,2652.50,0.0,0.0,-1,0,-1,-1,-1,БАШКОРТОСТАН РЕСП,1,1,0,1,0.0,1.0,4.0,-2.449294e-16,1.000000e+00,56.2667,54.9333,8.50,74.51,305.22,420.320,595.7650,848.127333,2281.51,1.0,0,351.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210871,1,2,2,1,44,639,609,30,2579.72,4630.13,157.72,788.61,5835.77,21,248.12,213.87,213.87,0.0,0.0,0,0,-1,-1,-1,СВЕРДЛОВСКАЯ ОБЛ,1,-1,0,0,0.0,1.0,4.0,-2.449294e-16,1.000000e+00,56.9053,59.9436,8.00,64.56,157.72,157.720,126.8525,217.529722,1104.05,1.0,1,66.0,1
1945410,0,1,3,1,46,152,58,0,3841.31,3841.31,859.05,4709.81,3801.80,3,39.50,39.50,39.50,0.0,0.0,-1,0,-1,-1,-1,None,1,1,0,1,0.0,1.0,4.0,-2.449294e-16,1.000000e+00,56.2353,68.7697,10.00,66.50,0.00,0.000,0.0000,0.000000,0.00,0.0,0,351.0,1
1202714,0,1,2,1,28,2183,33,2150,22.65,22.65,99.42,200.52,299.95,2,72.76,22.65,22.65,0.0,0.0,-1,0,-1,-1,-1,ЛИПЕЦКАЯ ОБЛ,-1,1,0,0,0.0,0.0,1.0,-2.449294e-16,1.000000e+00,52.3225,39.0850,8.00,64.56,14.40,8.140,12.5250,12.500000,14.40,0.0,0,354.0,1
260487,2,1,1,1,39,1477,164,29,487.80,5476.99,13219.51,162.60,14195.12,7,-0.00,-0.00,3525.77,4.0,4.0,-1,-1,-1,-1,-1,ВОЛОГОДСКАЯ ОБЛ,1,1,0,1,1.0,1.0,3.0,1.224647e-16,-1.000000e+00,58.6304,38.2367,11.50,60.43,162.60,6691.055,3345.5275,4460.703333,487.80,1.0,2,0.0,1


In [7]:
%cd /kaggle/working/

/kaggle/working


In [11]:
f1_scores = []
featrures_arr = []
for i in range(5):
    features = random.sample(list(X_train.columns), 40)
    featrures_arr.append(features)
    cat_cols = [col for col in features if X_train[col].dtype in ['object', 'category']]
    model = CatBoostClassifier(iterations=1000,
                               verbose=500,
                               learning_rate=0.02,
                               loss_function='Logloss', 
                               eval_metric='F1',
                               task_type='GPU',
                               depth=7,
                               class_weights=[1, 4],
                               random_seed=42)
    model.fit(X_train[features], y_train, cat_features=cat_cols , eval_set = (X_val[features], y_val))
    pred = model.predict(X_test[features])
    f1 = f1_score(y_test, pred)
    f1_scores.append(f1)
    model.save_model(f'model_{i}')
    
np.save('feature_part.npy', np.array(featrures_arr))
for i, f1 in enumerate(f1_scores):
    print(f'F1 score for model {i}: {f1}')

0:	learn: 0.5074018	test: 0.5084316	best: 0.5084316 (0)	total: 172ms	remaining: 2m 51s
500:	learn: 0.5536782	test: 0.5542490	best: 0.5542490 (500)	total: 1m 11s	remaining: 1m 11s
999:	learn: 0.5724251	test: 0.5682964	best: 0.5685292 (994)	total: 2m 21s	remaining: 0us
bestTest = 0.5685291907
bestIteration = 994
Shrink model to first 995 iterations.
0:	learn: 0.5114808	test: 0.5113141	best: 0.5113141 (0)	total: 174ms	remaining: 2m 54s
500:	learn: 0.5746119	test: 0.5711764	best: 0.5711764 (499)	total: 1m 12s	remaining: 1m 12s
999:	learn: 0.5964206	test: 0.5908080	best: 0.5908263 (998)	total: 2m 23s	remaining: 0us
bestTest = 0.5908262548
bestIteration = 998
Shrink model to first 999 iterations.
0:	learn: 0.4997648	test: 0.4980020	best: 0.4980020 (0)	total: 67.8ms	remaining: 1m 7s
500:	learn: 0.5631547	test: 0.5628955	best: 0.5629311 (498)	total: 12.5s	remaining: 12.4s
999:	learn: 0.5843168	test: 0.5811753	best: 0.5813519 (993)	total: 24.4s	remaining: 0us
bestTest = 0.5813519233
bestIterati

In [12]:
featrures_arr_2 = []
for i in range(5):
    features = random.sample(list(X.columns), 40)
    featrures_arr_2.append(features)
    cat_cols = [col for col in features if X[col].dtype in ['object', 'category']]
    model = CatBoostClassifier(iterations=1000,
                               verbose=500,
                               learning_rate=0.02,
                               loss_function='Logloss', 
                               eval_metric='F1',
                               task_type='GPU',
                               depth=7,
                               class_weights=[1, 4],
                               random_seed=42)
    model.fit(X[features], y, cat_features=cat_cols)
    model.save_model(f'model_full_{i}')
np.save('feature_full.npy', np.array(featrures_arr_2))

0:	learn: 0.5117690	total: 207ms	remaining: 3m 26s
500:	learn: 0.5735409	total: 1m 27s	remaining: 1m 27s
999:	learn: 0.5930979	total: 2m 52s	remaining: 0us
0:	learn: 0.5118918	total: 205ms	remaining: 3m 24s
500:	learn: 0.5734666	total: 1m 27s	remaining: 1m 26s
999:	learn: 0.5917044	total: 2m 52s	remaining: 0us
0:	learn: 0.5066340	total: 209ms	remaining: 3m 29s
500:	learn: 0.5697918	total: 1m 27s	remaining: 1m 27s
999:	learn: 0.5921169	total: 2m 52s	remaining: 0us
0:	learn: 0.5112164	total: 207ms	remaining: 3m 27s
500:	learn: 0.5760498	total: 1m 28s	remaining: 1m 27s
999:	learn: 0.5946931	total: 2m 54s	remaining: 0us
0:	learn: 0.5187105	total: 202ms	remaining: 3m 21s
500:	learn: 0.5629768	total: 1m 27s	remaining: 1m 27s
999:	learn: 0.5824365	total: 2m 53s	remaining: 0us
